<a href="https://colab.research.google.com/github/Mohneesh09/Trend-Rafa/blob/main/best_trendline_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
API_KEY = "717311313634cf72e5c3b021728afe4e"

In [ ]:
from urllib.request import urlopen 
import json
ticker='TSLA'
url = "https://financialmodelingprep.com/api/v3/historical-price-full/{}?apikey={}".format(ticker, API_KEY)
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
data_json = json.loads(response.read())

print(data_json)

{'symbol': 'TSLA', 'historical': [{'date': '2023-02-21', 'open': 204.99, 'high': 209.71, 'low': 199.07, 'close': 202.25, 'adjClose': 202.25, 'volume': 117427110, 'unadjustedVolume': 117427110, 'change': -2.74, 'changePercent': -1.33665, 'vwap': 203.68, 'label': 'February 21, 23', 'changeOverTime': -0.0133665}, {'date': '2023-02-17', 'open': 199.985, 'high': 208.44, 'low': 197.5, 'close': 208.31, 'adjClose': 208.31, 'volume': 213738549, 'unadjustedVolume': 213738549, 'change': 8.325, 'changePercent': 4.16, 'vwap': 204.75, 'label': 'February 17, 23', 'changeOverTime': 0.0416}, {'date': '2023-02-16', 'open': 210.78, 'high': 217.65, 'low': 201.84, 'close': 202.04, 'adjClose': 202.039993, 'volume': 229558338, 'unadjustedVolume': 229586500, 'change': -8.74, 'changePercent': -4.15, 'vwap': 211.32, 'label': 'February 16, 23', 'changeOverTime': -0.0415}, {'date': '2023-02-15', 'open': 211.755, 'high': 214.66, 'low': 206.11, 'close': 214.24, 'adjClose': 214.240005, 'volume': 182016181, 'unadjust

In [ ]:
df=pd.DataFrame(data_json['historical'])
df=df[['date','close','open','high','low']]

In [ ]:
len(df)

1258

In [ ]:
#https://medium.com/@chris_42047/3-ways-to-smooth-the-price-waves-with-python-9a4e170aaf15
#Will use Nadaraya–Watson kernel regression for smoothing prices.
from statsmodels.nonparametric.kernel_regression import KernelReg
def smoothed_curve(df_close,bandwidth):
  kernel_reg= KernelReg([df_close.values],[df_close.index],var_type='c',bw=bandwidth)
  reg_result = kernel_reg.fit([df_close.index])
  smoothed_df_close = pd.Series(data=reg_result[0], index=df_close.index)

  return smoothed_df_close

In [ ]:
smoothed_df_close=smoothed_curve(df['close'],bandwidth='cv_ls')
smoothed_df_low=smoothed_curve(df['low'],bandwidth='cv_ls')
smoothed_df_high=smoothed_curve(df['high'],bandwidth='cv_ls')

In [ ]:
df1= df[958:1258]
smoothed_df_close=smoothed_df_close[958:1258]

In [ ]:
df.columns

Index(['date', 'close', 'open', 'high', 'low'], dtype='object')

In [ ]:
from scipy.signal import argrelextrema
def find_extrema(df):
    #Calculating local min and max
    smoothed_local_max = argrelextrema(df['high'].values, np.greater, order=15)[0]
    smoothed_local_min = argrelextrema(df['low'].values, np.less, order=15)[0]

    smoothed_max_list = []
    smoothed_min_list = []
    max_id = 0
    min_id = 0
    for i in range(df.index.start, df.index.stop+1, 1):
        if max_id < len(smoothed_local_max):
            if i-df.index.start == smoothed_local_max[max_id]:
                smoothed_max_list.append(df['high'][i])
                max_id += 1
            else:
                smoothed_max_list.append(np.nan)
        if min_id < len(smoothed_local_min):
            if i-df.index.start == smoothed_local_min[min_id]:
                smoothed_min_list.append(df['low'][i])
                min_id += 1
            else:
                smoothed_min_list.append(np.nan)

    return smoothed_max_list, smoothed_min_list

In [ ]:
smoothed_max_list, smoothed_min_list=find_extrema(df1)

In [ ]:
def cal_trendlines(values):
    if len(values) == 0:
        return 0, 0
    x = np.arange(1, len(values) + 1, 1)
    y = np.array(values)

    x_new = x[~np.isnan(y)]
    y_new = y[~np.isnan(y)]
    #print(len(x_new))
    m, c = np.polyfit(x_new, y_new, 1)
    
    #>Try Houge Transform  (backfeet)
    #Window size creation from ATR.(research)
    #Add RMSE value.
    return m, c, len(x_new)    #Slope and Intercept

In [ ]:
x = np.arange(1, len(smoothed_max_list) + 1, 1)
y = np.array(smoothed_max_list)

x_new_max = x[~np.isnan(y)]
y_new_max = y[~np.isnan(y)]

x_new_max=np.flip(x_new_max)
y_new_max=np.flip(y_new_max)

In [ ]:
x11 = np.arange(1, len(smoothed_min_list) + 1, 1)
y11 = np.array(smoothed_min_list)

x_new_min = x11[~np.isnan(y11)]
y_new_min = y11[~np.isnan(y11)]

x_new_min=np.flip(x_new_min)
y_new_min=np.flip(y_new_min)

In [ ]:
y_new_max

array([23.9993, 20.866 , 24.9153, 25.8307, 21.1227, 25.2993, 23.4667,
       19.7447])

In [ ]:
x_new_max

array([297, 246, 220, 185, 146, 100,  74,  21])

In [ ]:
x_new_min

array([274, 238, 191, 163, 140, 108,  88,  69,  28,   5])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
ans=100
for i in range(0,2):
  for j in range(i+2,len(x_new_max)):
    LR=LinearRegression()
    LR.fit(x_new_max[i:j+1].reshape(-1, 1),y_new_max[i:j+1].reshape(-1, 1))
    y_new1= LR.predict(x_new_max[i:j+1].reshape(-1, 1))
    temp=(mean_absolute_error(y_new_max[i:j+1].reshape(-1, 1),y_new1)/np.mean(y_new_max))*100
    print(temp)
    if(ans>temp):
      ans=temp
      values_max= LR.coef_[0][0],LR.intercept_[0]
      x_max=x_new_max[i:j+1]
      y_max=y_new_max[i:j+1]
      



6.934819623102935
6.152672912894445
7.751991689159745
7.754552863987323
7.021343831923349
7.9474928054053136
3.6838181521529947
9.45582153813818
8.31229487400432
7.814487633409538
8.980236089661537


In [ ]:
ans11=100
for i in range(0,2):
  for j in range(i+2,len(x_new_min)):
    LR=LinearRegression()
    LR.fit(x_new_min[i:j+1].reshape(-1, 1),y_new_min[i:j+1].reshape(-1, 1))
    y_new1= LR.predict(x_new_min[i:j+1].reshape(-1, 1))
    temp=(mean_absolute_error(y_new_min[i:j+1].reshape(-1, 1),y_new1)/np.mean(y_new_min))*100
    print(temp)
    if(ans11>temp):
      ans11=temp
      values_min= LR.coef_[0][0],LR.intercept_[0]
      x_min=x_new_min[i:j+1]
      y_min=y_new_min[i:j+1]

1.7775956853181663
5.744946016017479
5.587725577387566
8.051867151630097
6.97944071409201
7.005474995911251
7.520043049332621
8.446636762487007
4.204220727422801
3.4960397464194353
8.993873596551266
7.543577176103949
7.202591230352945
7.192612454276688
8.155569450426848


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
title = f"Trend Analysis"
symbol='SPY'
fig = make_subplots(rows=1, cols=1, subplot_titles=[title])

fig.add_trace(go.Scatter(x=smoothed_df_close.index-958, y=smoothed_df_close.values, line=dict(color="blue", width=1),name="Smoothed Close"), row=1, col=1)
fig.add_trace(go.Scatter(x=smoothed_df_low.index-958, y=smoothed_df_low.values, line=dict(color="orange", width=1),name="Smoothed Low"), row=1, col=1)
fig.add_trace(go.Scatter(x=smoothed_df_high.index-958, y=smoothed_df_high.values, line=dict(color="aquamarine", width=1),name="Smoothed High"), row=1, col=1)
 #Trend Lines
y_min_extrema= values_min[0]*(smoothed_df_close.index-958) + values_min[1]
fig.add_trace(go.Scatter(x=smoothed_df_close.index-958, y=y_min_extrema, line=dict(color="red", width=1),name="Min Extrema Trend"), row=1, col=1)

y_max_extrema = values_max[0]*(smoothed_df_close.index-958) + values_max[1]
fig.add_trace(go.Scatter(x=smoothed_df_close.index-958, y=y_max_extrema, line=dict(color="green", width=1),name="Max Extrema Trend"), row=1, col=1)

#x = np.arange(0, len(smoothed_min_list),1)
fig.add_trace(go.Scatter(x=x_min, y=y_min, marker=dict(color='red', size=7), mode='markers', name='Min extrema'), row=1, col=1)
fig.add_trace(go.Scatter(x=x_max, y=y_max, marker=dict(color='green', size=7), mode='markers', name='Max extrema'), row=1, col=1)

fig.update_layout(
          title={'text': f"{symbol} Price", 'x': 0.5},
          autosize=True,
          width=1000, height=700,
          xaxis={"rangeslider": {"visible": False}},
          plot_bgcolor="#EFF5F8")
fig.update_yaxes(visible=False, secondary_y=True)

fig.show()

In [ ]:
#Prepare for Notebook Execution.
#Rolling function Code awaiting.
#Implement RANSAC. (Test on tickers sent)

In [ ]:
title = f"Trend Analysis"
symbol='AMD'
fig = make_subplots(rows=1, cols=1, subplot_titles=[title])

fig.add_trace(go.Scatter(x=smoothed_df_close.index-958, y=smoothed_df_close.values, line=dict(color="blue", width=1),name="Smoothed Close"), row=1, col=1)
fig.add_trace(go.Scatter(x=smoothed_df_low.index-958, y=smoothed_df_low.values, line=dict(color="orange", width=1),name="Smoothed Low"), row=1, col=1)
fig.add_trace(go.Scatter(x=smoothed_df_high.index-958, y=smoothed_df_high.values, line=dict(color="aquamarine", width=1),name="Smoothed High"), row=1, col=1)
 #Trend Lines
y_min_extrema= values_min[0]*(smoothed_df_close.index-958) + values_min[1]
fig.add_trace(go.Scatter(x=smoothed_df_close.index-958, y=y_min_extrema, line=dict(color="red", width=1),name="Min Extrema Trend"), row=1, col=1)

y_max_extrema = values_max[0]*(smoothed_df_close.index-958) + values_max[1]
fig.add_trace(go.Scatter(x=smoothed_df_close.index-958, y=y_max_extrema, line=dict(color="green", width=1),name="Max Extrema Trend"), row=1, col=1)

#x = np.arange(0, len(smoothed_min_list),1)
fig.add_trace(go.Scatter(x=x_min, y=y_min, marker=dict(color='red', size=7), mode='markers', name='Min extrema'), row=1, col=1)
fig.add_trace(go.Scatter(x=x_max, y=y_max, marker=dict(color='green', size=7), mode='markers', name='Max extrema'), row=1, col=1)

fig.update_layout(
          title={'text': f"{symbol} Price", 'x': 0.5},
          autosize=True,
          width=1000, height=700,
          xaxis={"rangeslider": {"visible": False}},
          plot_bgcolor="#EFF5F8")
fig.update_yaxes(visible=False, secondary_y=True)

fig.show()

In [ ]:
title = f"Trend Analysis"
symbol='TSLA'
fig = make_subplots(rows=1, cols=1, subplot_titles=[title])

fig.add_trace(go.Scatter(x=smoothed_df_close.index-958, y=smoothed_df_close.values, line=dict(color="blue", width=1),name="Smoothed Close"), row=1, col=1)
fig.add_trace(go.Scatter(x=smoothed_df_low.index-958, y=smoothed_df_low.values, line=dict(color="orange", width=1),name="Smoothed Low"), row=1, col=1)
fig.add_trace(go.Scatter(x=smoothed_df_high.index-958, y=smoothed_df_high.values, line=dict(color="aquamarine", width=1),name="Smoothed High"), row=1, col=1)
 #Trend Lines
y_min_extrema= values_min[0]*(smoothed_df_close.index-958) + values_min[1]
fig.add_trace(go.Scatter(x=smoothed_df_close.index-958, y=y_min_extrema, line=dict(color="red", width=1),name="Min Extrema Trend"), row=1, col=1)

y_max_extrema = values_max[0]*(smoothed_df_close.index-958) + values_max[1]
fig.add_trace(go.Scatter(x=smoothed_df_close.index-958, y=y_max_extrema, line=dict(color="green", width=1),name="Max Extrema Trend"), row=1, col=1)

#x = np.arange(0, len(smoothed_min_list),1)
fig.add_trace(go.Scatter(x=x_min, y=y_min, marker=dict(color='red', size=7), mode='markers', name='Min extrema'), row=1, col=1)
fig.add_trace(go.Scatter(x=x_max, y=y_max, marker=dict(color='green', size=7), mode='markers', name='Max extrema'), row=1, col=1)

fig.update_layout(
          title={'text': f"{symbol} Price", 'x': 0.5},
          autosize=True,
          width=1000, height=700,
          xaxis={"rangeslider": {"visible": False}},
          plot_bgcolor="#EFF5F8")
fig.update_yaxes(visible=False, secondary_y=True)

fig.show()